In [14]:
import numpy as np
import pandas as pd

In [60]:
df = pd.read_csv('pollution_dataset.csv')
states = ['Delhi', 'Uttarakhand', 'Haryana']
df = df[df['state'].isin(states)]
# df = df[df['state'] == 'Delhi' or df['state'] == 'Uttarakhand' or df['state'] ==]
df.drop_duplicates(inplace=True)
drop_col = list(df.columns)
drop_col.remove('state')
drop_col.remove('rspm')
drop_col.remove('spm')
drop_col.remove('so2')
drop_col.remove('no2')

df.drop(columns=drop_col, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,state,so2,no2,rspm,spm
0,Delhi,8.6,16.2,NaN,323.0
1,Delhi,6.5,9.3,NaN,488.0
2,Delhi,25.8,10.4,NaN,487.0
3,Delhi,0.5,2.8,NaN,528.0
4,Delhi,2.5,12.8,NaN,506.0


In [72]:
df.to_csv('original.csv')

In [39]:
df.shape

(13916, 5)

In [40]:
test = df.dropna(axis=0, how='any')
test.reset_index(drop=True, inplace=True)
test.shape

(4433, 5)

In [73]:
test.to_csv('withoutNull.csv')

In [42]:
def cal_SOi(so2):
    si=0
    if (so2<=40):
     si= so2*(50/40)
    elif (so2>40 and so2<=80):
     si= 50+(so2-40)*(50/40)
    elif (so2>80 and so2<=380):
     si= 100+(so2-80)*(100/300)
    elif (so2>380 and so2<=800):
     si= 200+(so2-380)*(100/420)
    elif (so2>800 and so2<=1600):
     si= 300+(so2-800)*(100/800)
    elif (so2>1600):
     si= 400+(so2-1600)*(100/800)
    return si
test['SOi']=test['so2'].apply(cal_SOi)

def cal_Noi(no2):
    ni=0
    if(no2<=40):
     ni= no2*50/40
    elif(no2>40 and no2<=80):
     ni= 50+(no2-40)*(50/40)
    elif(no2>80 and no2<=180):
     ni= 100+(no2-80)*(100/100)
    elif(no2>180 and no2<=280):
     ni= 200+(no2-180)*(100/100)
    elif(no2>280 and no2<=400):
     ni= 300+(no2-280)*(100/120)
    else:
     ni= 400+(no2-400)*(100/120)
    return ni
test['NOi']=test['no2'].apply(cal_Noi)

def cal_RSPMi(rspm):
    rpi=0
    if(rspm<=100):
     rpi = rspm
    elif(rspm>=101 and rspm<=150):
     rpi= 101+(rspm-101)*((200-101)/(150-101))
    elif(rspm>=151 and rspm<=350):
     rpi= 201+(rspm-151)*((300-201)/(350-151))
    elif(rspm>=351 and rspm<=420):
     rpi= 301+(rspm-351)*((400-301)/(420-351))
    elif(rspm>420):
     rpi= 401+(rspm-420)*((500-401)/(420-351))
    return rpi
test['RSPMi']=test['rspm'].apply(cal_RSPMi)

def cal_SPMi(spm):
    spi=0
    if(spm<=50):
     spi=spm*50/50
    elif(spm>50 and spm<=100):
     spi=50+(spm-50)*(50/50)
    elif(spm>100 and spm<=250):
     spi= 100+(spm-100)*(100/150)
    elif(spm>250 and spm<=350):
     spi=200+(spm-250)*(100/100)
    elif(spm>350 and spm<=430):
     spi=300+(spm-350)*(100/80)
    else:
     spi=400+(spm-430)*(100/430)
    return spi
test['SPMi']=test['spm'].apply(cal_SPMi)

def cal_aqi(si,ni,rspmi,spmi):
    aqi=0
    if(si>ni and si>rspmi and si>spmi):
     aqi=si
    if(ni>si and ni>rspmi and ni>spmi ):
     aqi=ni
    if(rspmi>si and rspmi>ni and rspmi>spmi ):
     aqi=rspmi
    if(spmi>si and spmi>ni and spmi>rspmi):
     aqi=spmi
    return aqi
test['AQI']=test.apply(lambda x:cal_aqi(x['SOi'],x['NOi'],x['RSPMi'],x['SPMi']),axis=1)

test.head()

,state,so2,no2,rspm,spm,SOi,NOi,RSPMi,SPMi,AQI
0,Delhi,11.0,39.0,184.0,405.0,13.75,48.75,217.417085,368.75,368.75
1,Delhi,12.0,42.0,190.0,429.0,15.00,52.50,220.402010,398.75,398.75
2,Delhi,14.0,41.0,187.0,304.0,17.50,51.25,218.909548,254.00,254.00
3,Delhi,13.0,39.0,192.0,323.0,16.25,48.75,221.396985,273.00,273.00
4,Delhi,12.0,36.0,170.0,325.0,15.00,45.00,210.452261,275.00,275.00


In [67]:
#KNN Imputation
from fancyimpute import KNN
ds = df[df['state']==states[2]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()
knn_imputer = KNN()
knn_res = knn_imputer.fit_transform(ds)
knn_res = pd.DataFrame(knn_res)
knn_res.columns = ds.columns
knn_res.head()

Imputing row 1/3415 with 2 missing, elapsed time: 2.039
Imputing row 101/3415 with 2 missing, elapsed time: 2.045
Imputing row 201/3415 with 1 missing, elapsed time: 2.051
Imputing row 301/3415 with 1 missing, elapsed time: 2.055
Imputing row 401/3415 with 0 missing, elapsed time: 2.058
Imputing row 501/3415 with 0 missing, elapsed time: 2.058
Imputing row 601/3415 with 0 missing, elapsed time: 2.058
Imputing row 701/3415 with 0 missing, elapsed time: 2.059
Imputing row 801/3415 with 0 missing, elapsed time: 2.059
Imputing row 901/3415 with 2 missing, elapsed time: 2.066
Imputing row 1001/3415 with 2 missing, elapsed time: 2.072
Imputing row 1101/3415 with 0 missing, elapsed time: 2.075
Imputing row 1201/3415 with 0 missing, elapsed time: 2.075
Imputing row 1301/3415 with 0 missing, elapsed time: 2.077
Imputing row 1401/3415 with 0 missing, elapsed time: 2.080
Imputing row 1501/3415 with 0 missing, elapsed time: 2.081
Imputing row 1601/3415 with 4 missing, elapsed time: 2.083
Imputing 

,so2,no2,rspm,spm
0,43.2,24.100000,73.356759,330.966334
1,41.8,24.000000,78.357400,362.720456
2,45.7,24.400000,84.000045,228.000000
3,42.8,22.000000,78.648075,364.839576
4,46.8,32.535293,262.309674,279.876996


In [98]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

ind = 2

ds = df[df['state']==states[ind]]
# ds.head()
ds.drop(columns=['state'], inplace=True)
# ds.isnull().sum()

MICE_imputer = IterativeImputer()
MICE_res = MICE_imputer.fit_transform(ds)
MICE_res = pd.DataFrame(MICE_res)
MICE_res.columns=ds.columns
MICE_res.head()

,so2,no2,rspm,spm
0,43.2,24.100000,252.515978,347.244579
1,41.8,24.000000,248.022287,344.286054
2,45.7,24.400000,175.915265,228.000000
3,42.8,22.000000,250.182195,336.278302
4,46.8,37.942087,270.185467,419.053661


In [99]:
MICE_res.isnull().sum()

so2     0
no2     0
rspm    0
spm     0
dtype: int64

In [100]:
ds.isnull().sum()

so2      281
no2      407
rspm     493
spm     1628
dtype: int64

In [101]:
MICE_res.to_csv(states[ind] + '_MICE.csv')